In [0]:
from pyspark.sql.functions import col, count, when
from pyspark.sql.types import FloatType
from pyspark.sql import functions as F

df = spark.table('dilshad_shawki.test.wine_reviews_data')
display(df.limit(5))

# Show schema
print('Schema:')
df.printSchema()

# Null counts for key columns
key_cols = ['country', 'description', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']
def null_count_expr(c):
    return count(when((col(c).isNull()) | (col(c) == ''), c)).alias(c)
null_counts = df.select([null_count_expr(c) for c in key_cols])
display(null_counts)

# UDF to safely convert to float
from pyspark.sql.functions import udf

def safe_float(x):
    try:
        return float(x)
    except:
        return None
safe_float_udf = udf(safe_float, FloatType())

safe_df = df.withColumn('points_float', safe_float_udf(col('points')))
safe_df = safe_df.withColumn('price_float', safe_float_udf(col('price')))

stats = safe_df.select(
    F.min('points_float').alias('min_points'),
    F.max('points_float').alias('max_points'),
    F.avg('points_float').alias('avg_points'),
    F.min('price_float').alias('min_price'),
    F.max('price_float').alias('max_price'),
    F.avg('price_float').alias('avg_price')
)
display(stats)

In [0]:
%restart_python


In [0]:
# Aggregate review counts, average points, and average price by country and variety
trend_df = df.withColumn('points_float', safe_float_udf(col('points')))
trend_df = trend_df.withColumn('price_float', safe_float_udf(col('price')))

country_trends = trend_df.groupBy('country').agg(
    F.count('*').alias('review_count'),
    F.avg('points_float').alias('avg_points'),
    F.avg('price_float').alias('avg_price')
).orderBy(F.desc('review_count'))
display(country_trends)

variety_trends = trend_df.groupBy('variety').agg(
    F.count('*').alias('review_count'),
    F.avg('points_float').alias('avg_points'),
    F.avg('price_float').alias('avg_price')
).orderBy(F.desc('review_count'))
display(variety_trends)

In [0]:
# Test if display works with a simple DataFrame
from pyspark.sql import Row
simple_df = spark.createDataFrame([Row(a=1, b=2), Row(a=3, b=4)])
display(simple_df)

In [0]:
%pip install textblob

In [0]:
%restart_python

In [0]:
# Sentiment analysis using TextBlob
from textblob import TextBlob
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

def get_sentiment(text):
    if text:
        return TextBlob(text).sentiment.polarity
    return None
sentiment_udf = udf(get_sentiment, FloatType())

sentiment_df = df.withColumn('sentiment', sentiment_udf(col('description')))

# Aggregate average sentiment by country and variety
country_sentiment = sentiment_df.groupBy('country').agg(F.avg('sentiment').alias('avg_sentiment')).orderBy(F.desc('avg_sentiment')).limit(10)
display(country_sentiment)

variety_sentiment = sentiment_df.groupBy('variety').agg(F.avg('sentiment').alias('avg_sentiment')).orderBy(F.desc('avg_sentiment')).limit(10)
display(variety_sentiment)

In [0]:
# Add price bands for analysis
import pyspark.sql.functions as F

price_bins = [0, 15, 30, 50, 100, 1000, 10000]
price_labels = ['<15', '15-30', '30-50', '50-100', '100-1000', '1000+']

from pyspark.sql.functions import when

sentiment_df = df.withColumn('points_float', safe_float_udf(col('points')))
sentiment_df = sentiment_df.withColumn('price_float', safe_float_udf(col('price')))
sentiment_df = sentiment_df.withColumn('sentiment', sentiment_udf(col('description')))

# Create price band column
sentiment_df = sentiment_df.withColumn(
    'price_band',
    when(col('price_float') < 15, '<15')
    .when((col('price_float') >= 15) & (col('price_float') < 30), '15-30')
    .when((col('price_float') >= 30) & (col('price_float') < 50), '30-50')
    .when((col('price_float') >= 50) & (col('price_float') < 100), '50-100')
    .when((col('price_float') >= 100) & (col('price_float') < 1000), '100-1000')
    .when(col('price_float') >= 1000, '1000+')
    .otherwise('Unknown')
)

# Aggregate by country and price band
country_price = sentiment_df.groupBy('country', 'price_band').agg(
    F.count('*').alias('review_count'),
    F.avg('points_float').alias('avg_points'),
    F.avg('sentiment').alias('avg_sentiment')
).orderBy(F.desc('review_count'))
display(country_price)

# Aggregate by variety and price band
variety_price = sentiment_df.groupBy('variety', 'price_band').agg(
    F.count('*').alias('review_count'),
    F.avg('points_float').alias('avg_points'),
    F.avg('sentiment').alias('avg_sentiment')
).orderBy(F.desc('review_count'))
display(variety_price)

# Aggregate by region_1 and price band
region_price = sentiment_df.groupBy('region_1', 'price_band').agg(
    F.count('*').alias('review_count'),
    F.avg('points_float').alias('avg_points'),
    F.avg('sentiment').alias('avg_sentiment')
).orderBy(F.desc('review_count'))
display(region_price)

In [0]:
# Identify best and worst wines by points, sentiment, and price-to-quality ratio
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# Add numeric columns if not already present
scored_df = df.withColumn('points_float', safe_float_udf(col('points')))
scored_df = scored_df.withColumn('price_float', safe_float_udf(col('price')))
scored_df = scored_df.withColumn('sentiment', sentiment_udf(col('description')))

# Price-to-quality ratio (points per dollar)
scored_df = scored_df.withColumn('ptq_ratio', F.col('points_float') / F.col('price_float'))

# Best wines: top by points, sentiment, and ptq_ratio
best_points = scored_df.orderBy(F.desc('points_float')).select('title', 'country', 'variety', 'points_float', 'price_float', 'sentiment', 'ptq_ratio').limit(10)
display(best_points)

best_sentiment = scored_df.orderBy(F.desc('sentiment')).select('title', 'country', 'variety', 'points_float', 'price_float', 'sentiment', 'ptq_ratio').limit(10)
display(best_sentiment)

best_ptq = scored_df.orderBy(F.desc('ptq_ratio')).select('title', 'country', 'variety', 'points_float', 'price_float', 'sentiment', 'ptq_ratio').limit(10)
display(best_ptq)

# Worst wines: bottom by points, sentiment, and ptq_ratio (filter for reasonable price and points)
worst_points = scored_df.filter(F.col('points_float') > 0).orderBy('points_float').select('title', 'country', 'variety', 'points_float', 'price_float', 'sentiment', 'ptq_ratio').limit(10)
display(worst_points)

worst_sentiment = scored_df.orderBy('sentiment').select('title', 'country', 'variety', 'points_float', 'price_float', 'sentiment', 'ptq_ratio').limit(10)
display(worst_sentiment)

worst_ptq = scored_df.filter(F.col('price_float') > 0).orderBy('ptq_ratio').select('title', 'country', 'variety', 'points_float', 'price_float', 'sentiment', 'ptq_ratio').limit(10)
display(worst_ptq)

**Wine Reviews Data Analysis: Final Review and Conclusion**

*Summary of Findings:*
* The dataset covers a wide range of wines, with the US, Italy, and France leading in review volume. France and Austria have the highest average points, while Pinot Noir and Bordeaux-style blends are top-rated varieties.
* Sentiment analysis shows Japan, Luxembourg, and South Korea have the most positively described wines, and rare varieties like Karasakiz and Moscato di Noto receive the most favorable language.
* Slicing by price bands reveals that higher-priced wines (50-100 and above) tend to score better in both points and sentiment, especially from regions like Napa Valley and Bordeaux.
* The best wines by points are Italian Merlot and Prugnolo Gentile, and French Chardonnay. By sentiment, US Chardonnay and Pinot Noir stand out. For value, US Merlot and Pinot Gris, and Portuguese Red offer the best price-to-quality ratios.
* The worst wines by points and sentiment are often low-cost or generic blends, with some expensive wines underperforming on a price-to-quality basis.

*Conclusion:*
The analysis highlights that while classic regions and varieties dominate in quality and reputation, there are hidden gems offering exceptional value and positive sentiment. Consumers seeking the best experience should consider both expert scores and review sentiment, and explore mid-to-high price bands for the best balance of quality and value. Conversely, caution is advised with generic blends and some high-priced wines that may not deliver on expectations.

*Actionable Insights:*
* For premium quality, focus on French, Italian, and US wines in the 50-100 price band, especially Pinot Noir, Bordeaux blends, and Chardonnay.
* For value, seek out US Merlot, Pinot Gris, and Portuguese Red in the lower price bands.
* Use sentiment trends to discover emerging regions and varieties with strong consumer appeal.

This comprehensive analysis provides a data-driven guide for wine selection, balancing expert ratings, consumer sentiment, and value.